# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [38]:
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd

In [36]:
from IPython.core.display import HTML
css = open('../../Stylesheets/style-table.css').read() + open('../../Stylesheets/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
document = ET.parse( './data/mondial_database.xml' )

****
__ Answer 1 - 10 countries with the lowest infant mortality rates__
****

In [196]:
mondial = document.getroot()
mondial.attrib

# Empty list to capture infant mortality
infant_mortality_by_country = []

#
city_population = []

ethnic_groups = []

for child in mondial.iterfind('country'):
    country_population_dict = {} 
    country_population_year = 0
    country = child.find('name').text
    for country_pop in child.iterfind('population'):
        country_population_dict[country_pop.attrib['year']] = int(country_pop.text)
    latest_year = max(country_population_dict.keys()) 
    latest_year_population = country_population_dict[latest_year]
    if(child.find('infant_mortality') is not None):
        # Convert text to float is important
        infant_mortality_value = float(child.find('infant_mortality').text)
        infant_mortality_by_country.append((country, infant_mortality_value))    
    else:
        infant_mortality_by_country.append((country, np.NaN))
    for ethnic_group in child.iterfind('ethnicgroup'):
        ethnic_groups.append((country, ethnic_group.text, float(ethnic_group.attrib['percentage']), 
                              latest_year, latest_year_population))
    for city in child.iterfind('city'):
        city_name = city.find('name').text
        measurement_year = 0
        for pop in city.iterfind('population'):
            population = pop.text
            year = pop.attrib['year']
            city_population.append((country, city_name, int(year), int(population)))

[('Albania', 'Albanian', 95.0, '2011', 2800138),
 ('Albania', 'Greek', 3.0, '2011', 2800138),
 ('Greece', 'Greek', 93.0, '2011', 10816286),
 ('Macedonia', 'Macedonian', 64.2, '2011', 2059794),
 ('Macedonia', 'Albanian', 25.2, '2011', 2059794),
 ('Macedonia', 'Turkish', 3.9, '2011', 2059794),
 ('Macedonia', 'Gypsy', 2.7, '2011', 2059794),
 ('Macedonia', 'Serb', 1.8, '2011', 2059794),
 ('Serbia', 'Serb', 82.9, '2011', 7120666),
 ('Serbia', 'Montenegrin', 0.9, '2011', 7120666),
 ('Serbia', 'Hungarian', 3.9, '2011', 7120666),
 ('Serbia', 'Roma', 1.4, '2011', 7120666),
 ('Serbia', 'Bosniak', 1.8, '2011', 7120666),
 ('Serbia', 'Croat', 1.1, '2011', 7120666),
 ('Montenegro', 'Montenegrin', 43.0, '2011', 620029),
 ('Montenegro', 'Serb', 32.0, '2011', 620029),
 ('Montenegro', 'Bosniak', 8.0, '2011', 620029),
 ('Montenegro', 'Albanian', 5.0, '2011', 620029),
 ('Kosovo', 'Albanian', 92.0, '2011', 1733872),
 ('Kosovo', 'Serbian', 5.0, '2011', 1733872),
 ('Andorra', 'Spanish', 43.0, '2011', 78115),

In [178]:
mortality = pd.DataFrame(infant_mortality_by_country, columns=['country', 'infant_mortality'])
mortality.sort_values(by='infant_mortality', ascending=True)[:10]

,country,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


****
__Answer 2 - 10 cities with the largest population__
****

In [174]:
pop_df = pd.DataFrame(city_population, columns=['country', 'city', 'year', 'population'])
pop_df.head()

,country,city,year,population
0,Albania,Tirana,1987,192000
1,Albania,Tirana,1990,244153
2,Albania,Tirana,2011,418495
3,Albania,Shkodër,1987,62000
4,Albania,Shkodër,2011,77075


In [177]:
city_group = pop_df.groupby('city')

max_city = city_group.max()
max_city = max_city.reset_index('city')
max_city.sort_values(by='population', ascending=False)[:10]

,city,country,year,population
315,Seoul,South Korea,2010,10229262
16,Al Qahirah,Egypt,2006,8471859
43,Bangkok,Thailand,1999,7506700
158,Hong Kong,Hong Kong,2009,7055071
157,Ho Chi Minh,Vietnam,2009,5968384
321,Singapore,Singapore,2010,5076700
9,Al Iskandariyah,Egypt,2006,4123869
253,New Taipei,Taiwan,2012,3939305
74,Busan,South Korea,2010,3813814
285,Pyongyang,North Korea,2008,3255288


****
__Answer 3 - 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)__
****

In [334]:
ethgroup_df = pd.DataFrame(ethnic_groups, columns=['country', 'ethnicity', 'percentage', 'year', 'country_population'])
ethgroup_df['ethnic_population'] = (ethgroup_df.percentage * ethgroup_df.country_population) // 100

ethgroup_df = ethgroup_df.sort_values(by='ethnicity', ascending=False)

ethgroup_df = ethgroup_df.groupby('ethnicity').sum().sort_values(by='ethnic_population', ascending=False).reset_index()
ethgroup_df = ethgroup_df[['ethnicity', 'ethnic_population']]
ethgroup_df[:10]

# = ethgroup_df.groupby('ethnicity', as_index=True)
#print(ethgroups.count().reset_index())

,ethnicity,ethnic_population
0,Han Chinese,1245058800
1,Indo-Aryan,871815583
2,European,494872201
3,African,318325104
4,Dravidian,302713744
5,Mestizo,157734349
6,Bengali,146776916
7,Russian,131856989
8,Japanese,126534212
9,Malay,121993548


****
__Answer 4 - name and country of a) longest river, b) largest lake and c) airport at highest elevation__
****

In [331]:
mondial2 = document.getroot()



In [ ]:
# Build a dict of all country codes

car_code = {}

for country in mondial2.iterfind('country'):
    car_code[country.attrib['car_code']] = country.find('name').text

In [332]:
rivers = []

# Create list of all river attribs that we care about
for river in mondial2.iterfind('river'):
    riverName = river.find('name').text if (river.find('name').text != None) else np.NaN
    riverLength = float(river.find('length').text) if (river.find('length') != None) else np.NaN
    riverCountries = river.attrib['country'].split(' ')
    
    # Convert all codes into the country names
    riverCountries = [car_code[x] for x in riverCountries]
    rivers.append((riverName, riverLength, riverCountries))

# Import rivers list into a dataframe
rivers_df = pd.DataFrame(rivers, columns=['name', 'length', 'countries'])

# Sort and pick the longest
rivers_df.sort_values(by='length', ascending=False)[['name', 'countries']][:1]

,name,countries
174,Amazonas,"[Colombia, Brazil, Peru]"


In [342]:
lakes = []

# Create list of all lake attribs that we care about
for lake in mondial2.iterfind('lake'):
    lakeName = lake.find('name').text if (lake.find('name').text != None) else np.NaN
    lakeArea = float(lake.find('area').text) if (lake.find('area') != None) else np.NaN
    lakeCountries = lake.attrib['country'].split(' ')
    
    # Convert all codes into the country names
    lakeCountries = [car_code[x] for x in lakeCountries]
    lakes.append((lakeName, lakeArea, lakeCountries))

# Import lakes list into a dataframe
lakes_df = pd.DataFrame(lakes, columns=['name', 'area', 'countries'])

# Sort and pick the longest
lakes_df.sort_values(by='area', ascending=False)[['name', 'area', 'countries']][:1]
    

,name,area,countries
54,Caspian Sea,386400,"[Russia, Azerbaijan, Kazakhstan, Iran, Turkmen..."


In [346]:
airports = []

# Create list of all lake attribs that we care about
for airport in mondial2.iterfind('airport'):
    airportName = airport.find('name').text if (airport.find('name').text != None) else np.NaN
    airportElevation = float(airport.find('elevation').text) if (airport.find('elevation').text != None) else np.NaN
    airportCountries = airport.attrib['country'].split(' ')
    
    # Convert all codes into the country names
    airportCountries = [car_code[x] for x in airportCountries]
    airports.append((airportName, airportElevation, airportCountries))

# Import lakes list into a dataframe
airports_df = pd.DataFrame(airports, columns=['name', 'elevation', 'countries'])

# Sort and pick the longest
airports_df.sort_values(by='elevation', ascending=False)[['name', 'elevation', 'countries']][:1]

,name,elevation,countries
80,El Alto Intl,4063,[Bolivia]
